# Generate UpSet plots for the COLO829 and NA12878 results

In [30]:
# Load the required packages
require(ggplot2)
require(vcfR)
require(UpSetR)
require(grid)

In [59]:
# Modify values for sample_name and sv_caller in order to generate a figure for a specific sample and SV caller callset
sample_name <- 'NA12878' #COLO829
#SV callers: 'delly', 'lumpy', 'gridss' or 'manta'
sv_caller <- 'delly'
workdir <- '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Papers/SV_callers'

# Name of the VCF file generated by SURVIVOR merge
filename <- paste(sample_name,sv_caller,'SURVIVOR.vcf',sep='_')
vcf_filepath <- file.path(workdir,sample_name, sv_caller, filename)

#Read the VCF file
vcf <- read.vcfR(vcf_filepath, verbose = FALSE)

In [ ]:
#Extract INFO->SUPP_VEC, separate each 0/1
mysplit_list <-
  strsplit(sapply(strsplit(sapply(strsplit(getFIX(vcf, getINFO = T)[, 'INFO'], ';')
                                  , function(x) {
                                    x[2]
                                  })
                           , '=')
                  , function(x) {
                    x[2]
                  }), '')

#Create a 0/1 matrix
mat <-
  matrix(0,
         ncol = length(mysplit_list[[1]]),
         nrow = length(mysplit_list))
for (i in 1:nrow(mat))
{
  mat[i, ] <- as.integer(mysplit_list[[i]])
}
#Get SVTYPE vector
svtype_vec <- getFIX(vcf)[, 'ALT']

In [ ]:
colnames(mat) <- c('run10',paste('run',1:9))
sv <- data.frame(ID = getFIX(vcf)[, 'ID'], mat, SVTYPE = svtype_vec)

## Show UpSet plot

In [ ]:
upset(
  sv,
  nsets = ncol(mat),
  sets.bar.color = "#56B4E9",
  order.by = "freq",
  text.scale = c(2, 2, 2, 2, 2, 2)
)
grid.text(paste(sample, sv_caller),
          x = 0.15,
          y = 0.95,
          gp = gpar(fontsize = 18))

In [ ]:
# Create folder for saving figures
dir.create('Figures')

## Save UpSet plot

In [ ]:
cairo_ps(paste("Figures/", sample_name, sv_caller, sep="_"), width = 10, height = 7)
upset(
  sv,
  nsets = ncol(mat),
  sets.bar.color = "#56B4E9",
  order.by = "freq",
  text.scale = c(2, 2, 2, 2, 2, 2)
)
grid.text(paste(sample_name, sv_caller),
          x = 0.15,
          y = 0.95,
          gp = gpar(fontsize = 18))
dev.off()

# Generate all figures

In [54]:
jaccard_df <- data.frame()

In [55]:
workdir <- '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Papers/SV_callers'

for(sample_name in c('NA12878', 'COLO829')){
    print(paste("Considering sample", sample_name))
    
    for(sv_caller in c('lumpy', 'delly', 'manta', 'gridss')){
        print(paste("Considering SV caller", sv_caller))
        
        filename <- paste(sample_name, sv_caller,'SURVIVOR.vcf',sep='_')
        vcf_filepath <- file.path(workdir,sample_name, sv_caller, filename)
        vcf <- read.vcfR(vcf_filepath, verbose = FALSE)
        
        mysplit_list <-
                  strsplit(sapply(strsplit(sapply(strsplit(getFIX(vcf, getINFO = T)[, 'INFO'], ';')
                                  , function(x) {
                                    x[2]
                                  })
                           , '=')
                  , function(x) {
                    x[2]
                  }), '')

        #Create a 0/1 matrix
        mat <-
          matrix(0,
                 ncol = length(mysplit_list[[1]]),
                 nrow = length(mysplit_list))
        for (i in 1:nrow(mat))
        {
          mat[i, ] <- as.integer(mysplit_list[[i]])
        }
        #Get SVTYPE vector
        svtype_vec <- getFIX(vcf)[, 'ALT']
        
        colnames(mat) <- c('run10',paste('run',1:9))
        sv <- data.frame(ID = getFIX(vcf)[, 'ID'], mat, SVTYPE = svtype_vec)
        
        cairo_ps(paste("Figures/", sample_name,"_",sv_caller,".eps", sep=""), width = 14, height = 7)
        upset(
          sv,
          nsets = ncol(mat),
          sets.bar.color = "#56B4E9",
          order.by = "freq",
          text.scale = c(2, 2, 2, 2, 2, 2)
        )
        grid.text(paste(sample_name, sv_caller),
                  x = 0.15,
                  y = 0.95,
                  gp = gpar(fontsize = 18))
        dev.off()
        
        jaccard_index = length(which(apply(mat,1,sum)==ncol(mat)))/length(which(apply(mat,1,sum)>0))*100
        jaccard_df <- rbind(jaccard_df, data.frame(sample=sample_name, sv_caller=sv_caller, jaccard_index=jaccard_index))
    }
}

[1] "Considering sample NA12878"
[1] "Considering SV caller lumpy"
[1] "Considering SV caller delly"
[1] "Considering SV caller manta"
[1] "Considering SV caller gridss"
[1] "Considering sample COLO829"
[1] "Considering SV caller lumpy"
[1] "Considering SV caller delly"
[1] "Considering SV caller manta"
[1] "Considering SV caller gridss"


In [56]:
workdir <- '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Papers/SV_callers'

for(sample_name in c('NA12878', 'COLO829')){
    print(paste("Considering sample", sample_name))
        
    for(run_n in 1:10){
        print(paste("Considering run", run_n))
        
        run <- paste('run', run_n,sep="")
        filename <- paste(sample_name, run,'SURVIVOR.vcf',sep='_')
        vcf_filepath <- file.path(workdir,sample_name, 'all', filename)
        vcf <- read.vcfR(vcf_filepath, verbose = FALSE)
        
        mysplit_list <-
                  strsplit(sapply(strsplit(sapply(strsplit(getFIX(vcf, getINFO = T)[, 'INFO'], ';')
                                  , function(x) {
                                    x[2]
                                  })
                           , '=')
                  , function(x) {
                    x[2]
                  }), '')

        #Create a 0/1 matrix
        mat <-
          matrix(0,
                 ncol = length(mysplit_list[[1]]),
                 nrow = length(mysplit_list))
        for (i in 1:nrow(mat))
        {
          mat[i, ] <- as.integer(mysplit_list[[i]])
        }
        #Get SVTYPE vector
        svtype_vec <- getFIX(vcf)[, 'ALT']
        
        colnames(mat) <- c('delly', 'gridss', 'lumpy', 'manta')
        sv <- data.frame(ID = getFIX(vcf)[, 'ID'], mat, SVTYPE = svtype_vec)
        
        cairo_ps(paste("Figures/", sample_name,"_",run,".eps", sep=""), width = 14, height = 7)
        upset(
          sv,
          nsets = ncol(mat),
          sets.bar.color = "#56B4E9",
          order.by = "freq",
          text.scale = c(2, 2, 2, 2, 2, 2)
        )
        grid.text(paste(sample_name, run),
                  x = 0.15,
                  y = 0.95,
                  gp = gpar(fontsize = 18))
        dev.off()
        
        jaccard_index = length(which(apply(mat,1,sum)==ncol(mat)))/length(which(apply(mat,1,sum)>0))*100
        jaccard_df <- rbind(jaccard_df, data.frame(sample=sample_name, sv_caller=run, jaccard_index=jaccard_index))
                            
    }
}

[1] "Considering sample NA12878"
[1] "Considering run 1"
[1] "Considering run 2"
[1] "Considering run 3"
[1] "Considering run 4"
[1] "Considering run 5"
[1] "Considering run 6"
[1] "Considering run 7"
[1] "Considering run 8"
[1] "Considering run 9"
[1] "Considering run 10"
[1] "Considering sample COLO829"
[1] "Considering run 1"
[1] "Considering run 2"
[1] "Considering run 3"
[1] "Considering run 4"
[1] "Considering run 5"
[1] "Considering run 6"
[1] "Considering run 7"
[1] "Considering run 8"
[1] "Considering run 9"
[1] "Considering run 10"


## Print and save the data.frame with Jaccard indices

In [57]:
print(jaccard_df)

    sample sv_caller jaccard_index
1  NA12878     lumpy  100.00000000
2  NA12878     delly   99.98690071
3  NA12878     manta   99.99458738
4  NA12878    gridss   99.49589966
5  COLO829     lumpy  100.00000000
6  COLO829     delly  100.00000000
7  COLO829     manta  100.00000000
8  COLO829    gridss   99.90921836
9  NA12878      run1    0.12444396
10 NA12878      run2    0.12443198
11 NA12878      run3    0.12444053
12 NA12878      run4    0.12444396
13 NA12878      run5    0.12443797
14 NA12878      run6    0.12443711
15 NA12878      run7    0.12443711
16 NA12878      run8    0.12443882
17 NA12878      run9    0.12443626
18 NA12878     run10    0.12444396
19 COLO829      run1    0.02592806
20 COLO829      run2    0.02592806
21 COLO829      run3    0.02592747
22 COLO829      run4    0.02592630
23 COLO829      run5    0.02592806
24 COLO829      run6    0.02592776
25 COLO829      run7    0.02592806
26 COLO829      run8    0.02592572
27 COLO829      run9    0.02592776
28 COLO829     run10

In [58]:
write.csv(file='Jaccard_index.csv', jaccard_df)